In [1]:
#load "llm.fsx"

open System
open TorchSharp

let device = Llm.device
printfn "Using device: %s" (device.ToString())

Using CUDA / GPU
Using device: cuda


In [2]:
printfn "%s" <| System.Environment.GetEnvironmentVariable("LD_LIBRARY_PATH")
printfn "%s" <| System.Runtime.InteropServices.RuntimeInformation.FrameworkDescription

/usr/local/cuda-11.7/lib64:
.NET 7.0.13


In [3]:
let text = IO.File.ReadAllText("shakespeare.txt")
let chars = Seq.distinct text |> Seq.toArray |> Array.sort
let vocabSize = chars.Length
printfn "Vocab size: %d" vocabSize
printfn "Chars: %s" (String.Join("", chars))

Vocab size: 65
Chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
let cfg = {|
    vocabSize = vocabSize
    blockSize = 32
    batchSize = 4L
    encodingSize = 80L
    nHeads = 4L
    dropout = 0.1
    bias = false
|}        


In [5]:
let encode, decode =
    let maxCharCode = chars |> Seq.map int |> Seq.max
    let char2index = Array.zeroCreate (maxCharCode + 1)
    let index2char = Array.zeroCreate (vocabSize + 1)
    for i,ch in Seq.indexed chars do
        char2index[int ch] <- int i
        index2char[i] <- ch
    let encode (s:string) =
        Array.init s.Length (fun i -> char2index[int s[i]])
    let decode (xs: int[]) =
        xs |> Array.map (fun i -> index2char[int i]) |> String.Concat
    encode, decode

printfn "hello world -> %A -> %A" (encode "hello world") (decode (encode "hello world"))

hello world -> [|46; 43; 50; 50; 53; 1; 61; 53; 56; 50; 42|] -> "hello world"


In [6]:
let trainData, validData =
    let splitIdx = text.Length * 9 / 10
    encode text[0..splitIdx-1], encode text[splitIdx..]
printfn "Shapes: train=%d, validation=%d" trainData.Length validData.Length


Shapes: train=1003854, validation=111540


In [7]:
type BatchGenerator (data: int[], blockSize, batchSize, device) =
    let nChars = data.Length
    let maxRnd = nChars - blockSize
    let mutable random = new Random()
    let xs = Array.zeroCreate (batchSize * blockSize)
    let ys = Array.zeroCreate (batchSize * blockSize)
    let tensorShape = [|int64 batchSize; int64 blockSize|]
    let mutable charsServed = 0
    member this.StartReproducibleRandom() =
        random <- new Random(1337)
    member this.GetBatch() =
        for i in 0..batchSize-1 do
            let ix = random.Next(maxRnd)
            let xdst = xs.AsSpan(i*blockSize, blockSize)
            let xsrc = data.AsSpan(ix, blockSize)
            xsrc.CopyTo(xdst)
            let ydst = ys.AsSpan(i*blockSize, blockSize)
            let ysrc = data.AsSpan(ix+1, blockSize)
            ysrc.CopyTo(ydst)

        charsServed <- charsServed + batchSize * blockSize

        let xb = torch.tensor(xs, dimensions=tensorShape, dtype=torch.int64, device=device, requires_grad=false)
        let yb = torch.tensor(ys, dimensions=tensorShape, dtype=torch.int64, device=device, requires_grad=false)
        xb, yb
    member this.Epoch = float charsServed / float nChars

let test() =
    let xb, yb = BatchGenerator([| for i in 0..12 -> int i |], 5, 4, device).GetBatch()
    xb.slice(0L, 0L, 1L, 1L), yb.slice(0L, 0L, 1L, 1L)
test()


Item1,"[1x5], type = Int64, device = cuda:0 3 4 5 6 7"
Item2,"[1x5], type = Int64, device = cuda:0 4 5 6 7 8"


In [8]:
let tr =
    //new CausalSelfAttention(4)
    (new Llm.Block(4, cfg.encodingSize, cfg.bias, cfg.dropout)).``to``(device)
let t = torch.rand([|1L;8L;int64 cfg.encodingSize|], device=device, dtype=torch.float32)
tr.forward(t)


[1x8x80], type = Float32, device = cuda:0
[0,..,..] =
 0.59362 1.4849 1.9901 0.049434 -0.77909 0.01703 0.83395 0.18871 0.39203 0.76778 -0.024441 ...
 0.095971 1.328 0.49838 0.088462 -0.49314 -0.27902 -0.19273 0.87237 -0.40437 0.33468 -0.080259 ...
 0.80089 0.46377 0.92359 0.83689 0.1874 -0.41342 1.0601 -0.16266 -0.26702 0.80272 1.2511 ...
 0.69727 0.40824 0.60477 -0.31747 -0.26855 -0.09311 0.43753 0.77788 0.18536 1.0276 -0.082679 ...
 0.55922 0.64551 1.2285 0.50303 0.42839 -0.048008 -0.016028 0.097681 -0.49408 0.33856 0.68956 ...
 0.73436 0.50622 1.1139 0.72106 -0.032265 0.40349 0.69118 0.98259 0.40161 0.63489 0.34039 ...
 0.62511 0.017181 1.1316 -0.0040646 -0.33799 0.10026 0.10848 1.0154 0.2851 1.0834 0.41103 ...
 1.1085 -0.094517 1.1696 0.40838 -0.088218 0.67409 0.49232 0.33916 0.1886 1.2699 -0.11017 ...

In [9]:
let loss (ys:torch.Tensor) (yHat:torch.Tensor) =
    let batchSize = ys.shape.[0]
    let logits = yHat.reshape(-1, vocabSize)
    let target = ys.reshape([|batchSize * (int64 cfg.blockSize)|])
    //printfn "shapes: logits=%A targets=%A" logits.shape target.shape
    let loss = torch.nn.functional.cross_entropy(logits, target)
    loss

let evalLosses : (torch.Tensor->torch.Tensor) -> float*float =
    let nLoops = 16
    let batchSize = 256
    let trainBatchGen = BatchGenerator(trainData, cfg.blockSize, batchSize, device)
    let validBatchGen = BatchGenerator(validData, cfg.blockSize, batchSize, device)
    fun (forward : torch.Tensor -> torch.Tensor) ->
        let getLoss (bg:BatchGenerator) =
            use _ = torch.NewDisposeScope()
            bg.StartReproducibleRandom()
            torch.stack([
                for i in 1..nLoops do
                    let xb,yb = bg.GetBatch()
                    yield loss yb (forward xb)
            ]).mean().ToDouble()

        getLoss trainBatchGen, getLoss validBatchGen

### Select which the model to train
... locally defined or the one loaded from script (both should have the same number of parameters)

In [10]:
let model =
    new Llm.LanguageModel(2, cfg.nHeads, cfg.encodingSize, cfg.vocabSize, cfg.blockSize, cfg.bias, cfg.dropout)
    //let model = torch.jit.load<torch.Tensor, torch.Tensor>("shakespeare.pt.zip")
model.``to``(device) |> ignore


Seq.sum [ for p in model.parameters() do if p.requires_grad then p.numel() ]
|> printfn "Number of parameters: %d"
//for n,p in model.named_parameters() do
//    printfn "%s: %A on %A" n p.shape p.device

if false then
    let xb, yb = BatchGenerator(trainData, cfg.blockSize, 16, device).GetBatch()
    printfn "xb: %A on %A" xb.shape xb.device
    let yHat = model.forward(xb) |> unbox<torch.Tensor>
    printfn "%A" yHat.shape


//loss yb yHat, evalLosses model
//loss yb yHat
//printfn "%s" <| model.generate 100 "       w"


Number of parameters: 154465


In [11]:
/// run inference to generate some text from the model
let generate (forward:torch.Tensor -> torch.Tensor) nTokens (input:string) =
    let blockSize = cfg.blockSize

    let generateIxs (ixs:torch.Tensor) nTokens =
        model.eval()
        let mutable ixs = ixs
        let result = [|
            yield ixs
            for i in 1..nTokens do
                //printfn "ixs.shape=%A" ixs.shape
                use yHat = model.forward(ixs)
                use yHat' = yHat.select(1L,-1L)
                use probs = torch.nn.functional.softmax(yHat', 1)
                //printfn "yHat.shape=%A" yHat.shape
                let yIdx = torch.multinomial(probs, num_samples=1)
                use ixs' =
                    if ixs.shape[1] >= blockSize then
                        ixs.slice(1L, 1L, int64 blockSize, 1L)
                    else
                        ixs.clone()
                ixs <- torch.cat([|ixs'; yIdx|], dim=1)
                //if i = 5 then
                //    printfn "shapes: ixs: %A yHat: %A yHat': %A probs: %A yIdx: %A" ixs.shape yHat.shape yHat'.shape probs.shape yIdx.shape
                yield yIdx
        |]
        torch.cat(result, dim=1)


    let input = input.PadLeft(blockSize, ' ')
    if input.Length <> blockSize then
        failwithf "Expected input of length %d, got %d" blockSize input.Length
    let idx = torch.tensor(encode input, dimensions=[|1L;int64 blockSize|], dtype=torch.int64, device=device, requires_grad=false)
    let idx = idx.expand([|2L;-1L|])
    //printfn "idx.shape=%A" idx.shape
    let idx' = generateIxs idx nTokens
    //printfn "generated.shape=%A" idx'.shape
    let idx' = idx'.select(0L,0L).cpu()
    let ords = Array.init (int idx'.NumberOfElements) (fun i -> idx'[int64 i].ToInt32())
    //idx'.to_type(torch.int32, copy=false).data().CopyTo(ords, 0, 0)
    decode ords

generate model.forward 100 "w" // initially this should render some jibberish

                               wRijm3R?BZ!cUWD!gB,XBHU'z.aW;GlUdY
GUKhaYREyQh,PZt,!ZJ?
,GA..gFSk!sQs,aQuSlhSSgnkdBG.EtArh$lwNWrxa?nH

### Select optimization features

In [12]:
printfn "flash sdp enabled = %b" (torch.backends.cuda.flash_sdp_enabled())

// use tf32 for matmul

//torch.backends.cuda.matmul.allow_tf32 <- true
torch.backends.cuda.matmul.allow_tf32 <- false

flash sdp enabled = true


In [13]:
let optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

let batchSize = 384
let batchGen = BatchGenerator(trainData, cfg.blockSize, batchSize, device)

let train nSteps =
    model.train()
    for step in 0..nSteps do
        use _ = torch.NewDisposeScope()
        let xb, yb = batchGen.GetBatch()
        let yHat = model.forward(xb)
        let loss = loss yb yHat
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() |> ignore
        if step % 5000 = 0 then
            model.eval()
            let trainLoss,validLoss = evalLosses model.forward
            printfn "Step %5d: loss=%6f train=%6f valid=%6f epoch=%.2f" step (loss.ToDouble()) trainLoss validLoss batchGen.Epoch
            model.train()



In [14]:
// pretrain a bit to warmup dotent optimizer
train 2000

GC.Collect()

Step     0: loss=4.556058 train=4.534179 valid=4.535075 epoch=0.01
Elapsed: 00:00:11.7786774


In [15]:
// do the actual training (measure time)

let stopwatch = Diagnostics.Stopwatch.StartNew()

train 50000

stopwatch.Stop()
printfn "Elapsed: %A" stopwatch.Elapsed


Step     0: loss=2.204069 train=2.179144 valid=2.205093 epoch=24.51
Step  5000: loss=1.862904 train=1.829885 valid=1.952835 epoch=85.71
Step 10000: loss=1.764589 train=1.721904 valid=1.879439 epoch=146.91
Step 15000: loss=1.701865 train=1.668274 valid=1.834176 epoch=208.12
Step 20000: loss=1.708186 train=1.634280 valid=1.805166 epoch=269.32
Step 25000: loss=1.669354 train=1.610204 valid=1.783962 epoch=330.53
Step 30000: loss=1.661941 train=1.589250 valid=1.767292 epoch=391.73
Step 35000: loss=1.623001 train=1.573976 valid=1.757389 epoch=452.93
Step 40000: loss=1.623608 train=1.560819 valid=1.748245 epoch=514.14
Step 45000: loss=1.596339 train=1.550826 valid=1.737085 epoch=575.34
Step 50000: loss=1.605995 train=1.541117 valid=1.728340 epoch=636.55
Elapsed: 00:04:53.4185397


In [16]:
printfn "%s" <| generate model.forward 500 "M"

                               Montague about like a'em them brother-back,
But men me to mean
that a puning; self.
The praye! this are depose love use a is offedings,
But so, the crow,
Recomes was the much pley? hope have pardo stay's noble think you but homefore,--

FRIAR PETER:
Look givession
coming me deads when us. But she come of him wakes my sistand,
Bot, with you ways; he must heads of cursame to much bed I eyes!

QUEEN ELIZABETH:
Go their what earth!

Julish'd this homeo unce: in to grace thee
Rillows in grad
You have 


In [21]:
model.named_parameters()

index value 0 (layers.embed.tok_emb.weight, [65x80], type = Float32, device = cuda:0) Item1 layers.embed.tok_emb.weight Item2 [65x80], type = Float32, device = cuda:0
 -0.50116 0.091951 0.19848 -0.084596 -0.50279 -1.3935 0.33737 0.72789 -0.044275 0.79807 -2.5852 ...
 0.22035 -1.3038 0.16243 1.4349 -0.61398 -0.15909 1.6727 -0.30105 1.0476 0.55523 -1.5434 ...
 1.8219 -0.77285 0.27537 0.047399 1.5596 0.13497 1.7296 -0.55037 0.41364 -0.40691 -0.37408 ...
 1.6552 0.57316 -0.70437 -1.6979 -0.69112 0.50357 0.31569 1.0437 1.0535 -1.8841 0.97331 ...
 -1.1638 0.69923 -0.40748 -1.4279 -0.91847 -1.0716 0.7583 -1.0169 1.9301 -1.5954 -1.4983 ...
 -0.46511 -0.96148 1.0982 1.7167 -1.0415 -0.035214 0.010604 1.4801 1.3114 -0.60903 0.59717 ...
 0.13294 0.18351 -0.54688 -0.099289 0.8268 -0.50858 -0.73767 0.42795 0.019657 -1.8232 0.41527 ...
 0.38849 -0.61156 -0.5782 0.88228 0.26037 0.0021067 -0.92304 -0.068625 -0.29503 -0.83198 -1.3138 ...
 -0.89183 -1.6022 -1.7331 -0.99106 0.01151 0.8443 -1.5925 -2.0171 0.67784 0.1138 -1.2953 ...
 1.8124 0.12848 -0.029058 0.44632 -0.37374 0.63578 -0.513 -2.1734 -1.1758 -0.63678 -0.82045 ...
 0.38846 -0.68877 -1.3722 0.80071 -0.79115 -0.0316 -0.06575 1.5523 -1.0451 -0.24606 -0.41156 ...
 1.5396 -0.87301 -0.84659 0.79086 2.3492 0.10205 1.1518 0.83952 0.22893 -1.3088 -1.0815 ...
 0.73863 0.48263 -0.39833 1.0127 0.23492 0.62729 1.2512 -1.69 -1.9859 -0.13747 0.39443 ...
 0.33083 -0.096799 -1.1613 -0.59772 1.8016 2.1467 -1.3123 0.73783 -0.53821 -1.1794 1.9047 ...
 -0.2677 -0.16159 -1.5748 -0.78754 -0.47201 -0.14225 1.1458 -0.56892 -0.98159 1.033 -0.51223 ...
 0.51776 -0.24304 1.5411 0.23486 1.0549 -0.035374 -2.3713 1.1935 -0.089191 -0.15227 -0.19219 ...
 0.45939 -0.58988 0.19052 1.2508 -1.38 0.69562 -0.51915 -2.3426 0.022597 1.3596 -0.95419 ...
 -0.096785 1.9355 1.9177 1.2341 0.23101 1.136 1.3759 -0.002971 -1.5736 1.741 -1.0561 ...
 0.20477 -0.60745 0.21156 1.0823 -0.020653 1.2823 -2.2846 -0.68119 0.80079 1.1812 0.51834 ...
 0.81006 -0.01114 0.57467 1.1048 -0.14855 1.2391 0.53758 -0.19994 1.1193 1.4202 1.3884 ...
 -0.034892 -0.59096 1.5285 -1.1762 0.77051 -2.0209 1.9814 0.93932 -0.33599 0.11177 -1.0042 ...
 0.33394 0.32266 -1.4356 0.38774 1.2701 -1.4463 2.3053 2.4521 0.23583 0.10064 0.57387 ...
 -0.86198 -1.0159 0.55361 -1.1941 0.61441 -0.62204 -1.267 -0.54369 -2.1035 -0.26555 1.12 ...
 0.37264 -1.8423 1.9508 -0.38511 -0.04843 -0.034883 -0.52716 -0.078626 1.427 1.9131 1.593 ...
 0.025233 -1.5581 0.57674 0.51633 -0.30995 0.017612 0.5682 0.097986 0.89255 0.2902 -0.80506 ...
 1.9327 -1.227 -0.29069 1.2574 0.93555 1.3186 -0.63004 -1.1093 0.94103 0.72209 0.35801 ...
 1.4645 0.020341 1.1819 1.1423 -0.77206 -0.85658 1.2806 0.0068735 -0.48755 0.61633 0.42006 ...
 0.56674 -0.097074 -1.011 -1.4141 -0.5593 0.25574 0.40223 -0.96096 0.98827 -1.9837 0.45618 ...
 0.99264 -0.84127 1.5304 3.6255 1.6821 2.6413 -0.0040883 -0.22149 1.7552 -0.79842 -0.040061 ...
 0.11718 -0.97069 1.2586 0.15752 1.469 0.048358 -2.1595 -1.4312 -1.7551 -0.52141 -0.11583 ...
 3.0484 -1.9115 -0.12164 -0.83862 -0.25318 0.037847 -2.1391 -1.733 0.88112 -0.81224 -0.30697 ...
 -0.71812 0.39781 -0.8653 -1.5525 0.72758 0.28413 -1.389 -1.7057 -0.03928 0.21043 -1.0425 ...
 -0.70221 -0.12848 0.46906 0.98982 -1.4966 0.68192 -0.29418 0.10415 -0.72909 2.901 0.3948 ...
 -0.22837 0.75361 -0.83414 -0.96761 -0.36305 1.7888 0.98791 0.45834 1.7479 -0.55004 -0.18335 ...
 -2.2762 -2.1364 -0.82752 1.6397 0.8589 0.3022 -1.2012 0.68656 -0.72914 1.1306 0.33597 ...
 1.3749 -1.3726 -0.26091 -0.38496 0.080216 -0.46211 -1.8126 1.1134 -0.29239 -1.3514 0.1861 ...
 -1.2544 -1.6248 -1.8351 0.72747 -0.9647 -0.42621 0.047128 1.8724 1.2195 -2.1369 -0.31651 ...
 0.040349 -0.61526 -0.045024 0.35396 -1.6154 -0.94258 0.15388 -1.1944 -1.9064 -0.48499 0.020207 ...
 1.7498 0.48 1.2931 0.93162 -1.2151 0.83012 0.49795 -0.74679 -0.59956 -0.51499 0.082394 ...
 -0.87206 -0.44722 -0.11511 0.51682 0.94588 0.65012 1.0737 0.18558 -0.44376 -0.28018 -1.3059 ...
 -0.5757 0.2275 -1.2999 -0.70703 -0.043278 2.0419 -1